# GA Optimization

In [81]:
ga_accs={"CNN":0,
         "ELU":0,
         "MLP":0,
         "RNN":0,}


In [84]:
import pickle
from sklearn.preprocessing import OneHotEncoder
import numpy as np 
with open('X.pickle', 'rb') as file:
    X = pickle.load(file)

with open('y.pickle', 'rb') as file:
    y = pickle.load(file)

print(np.shape(X))
print(np.shape(y))

# X=np.reshape(X,((2592, 8,1)))
input_shape = np.shape(X)
num_classes = 4

print("Input shape:", input_shape)
print("Number of classes:", num_classes)

X = X.reshape((X.shape[0], X.shape[1], 1, 1))
input_shape = X.shape[1:]  # (8, 1, 1)
print("Input shape:", input_shape)

(2592, 8)
(2592, 4)
Input shape: (2592, 8)
Number of classes: 4
Input shape: (8, 1, 1)


# CNN

In [86]:
import random
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

search_space = {
    'num_conv_layers': range(1, 10),
    'num_filters': [16, 32, 64],
    'filter_sizes': [(3, 3), (5, 5),(7,7)],
    'num_dense_layers': range(1, 10),
    'dense_layer_units': [32, 64, 128, 256]
}
best_fitnesses=[]
population_size = 20
num_generations = 100
mutation_rate = 0.1

def evaluate_cnn_model(params, input_shape, num_classes, X_train, y_train, X_test, y_test):
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
    if input_shape[0] >= 2 and input_shape[1] >= 2:
        model.add(layers.MaxPooling2D((2, 2)))
    else:
        model.add(layers.MaxPooling2D((1, 1)))
    for _ in range(params['num_conv_layers']):
        model.add(layers.Conv2D(params['num_filters'], params['filter_sizes'], activation='relu', padding='same'))
        if model.output_shape[1] >= 2 and model.output_shape[2] >= 2:
            model.add(layers.MaxPooling2D((2, 2)))
        else:
            model.add(layers.MaxPooling2D((1, 1)))
    model.add(layers.Flatten())
    for _ in range(params['num_dense_layers']):
        model.add(layers.Dense(params['dense_layer_units'], activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=3)
    model.fit(x=X_train, y=y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=0, callbacks=[early_stopping])
    
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy



def initialize_population(population_size, search_space):
    population = []
    for _ in range(population_size):
        individual = {param: random.choice(values) for param, values in search_space.items()}
        population.append(individual)
    return population

def crossover(parent1, parent2):
    child = {}
    for param in parent1.keys():
        child[param] = random.choice([parent1[param], parent2[param]])
    return child

def mutate(individual, mutation_rate, search_space):
    for param in individual.keys():
        if random.random() < mutation_rate:
            individual[param] = random.choice(search_space[param])
    return individual

def genetic_algorithm(population_size, num_generations, mutation_rate, search_space, input_shape, num_classes, X_train, y_train, X_test, y_test):
    population = initialize_population(population_size, search_space)
    for generation in range(num_generations):
        print(f"Generation {generation + 1}/{num_generations}")
        fitness_scores = []
        for individual in population:
            fitness = evaluate_cnn_model(individual, input_shape, num_classes, X_train, y_train, X_test, y_test)
            fitness_scores.append((individual, fitness))
        fitness_scores.sort(key=lambda x: x[1], reverse=True)
        print(f"Best accuracy in generation {generation + 1}: {fitness_scores[0][1]}")
        selected_parents = [individual for individual, _ in fitness_scores[:2]]
        next_generation = selected_parents
        while len(next_generation) < population_size:
            parent1, parent2 = random.sample(selected_parents, 2)
            child = crossover(parent1, parent2)
            child = mutate(child, mutation_rate, search_space)
            next_generation.append(child)
        population = next_generation
    best_individual, best_fitness = max(fitness_scores, key=lambda x: x[1])
    best_fitnesses.append(best_individual)
    return best_individual, best_fitness

# input_shape = (22, 4, 1)
# num_classes = 4
split_index = int(0.8 * X.shape[0])  
X_train, X_test = tf.split(X, [split_index, X.shape[0] - split_index])
y_train, y_test = tf.split(y, [split_index, y.shape[0] - split_index])

best_individual, best_fitness = genetic_algorithm(population_size, num_generations, mutation_rate, search_space, input_shape, num_classes, X_train, y_train, X_test, y_test)
print("Best CNN Architecture found:")
print(best_individual)
print("Best Accuracy:", "%.2f" %best_fitness)
ga_accs["CNN"]=best_fitness

Generation 1/100
Best accuracy in generation 1: 0.39306357502937317
Generation 2/100
Best accuracy in generation 2: 0.38535645604133606
Generation 3/100
Best accuracy in generation 3: 0.3757225573062897
Generation 4/100
Best accuracy in generation 4: 0.3988439440727234
Generation 5/100


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(best_fitness)
plt.xlabel('Generation')
plt.ylabel('Best Fitness Score')
plt.title('Genetic Algorithm Convergence')
plt.show()

# EGG-Net

In [ ]:
import random
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, ELU, Dropout, Dense, Flatten
from keras import regularizers

search_space = {
    'num_conv_layers': range(1, 4),
    'num_filters': [16, 32, 64],
    'filter_sizes': [32, 64, 128],
    'num_dense_layers': range(1, 4),
    'dense_layer_units': [32, 64, 128]
}

population_size = 10
num_generations = 5
mutation_rate = 0.1

def build_classifier(params, input_shape, num_classes):
    classifier = Sequential()
    
    classifier.add(Conv2D(16, (3, 3), input_shape=input_shape, padding='same',
                          kernel_regularizer=regularizers.l2(0.01)))
    classifier.add(BatchNormalization())
    classifier.add(ELU())
    classifier.add(Dropout(0.5))
    
    for _ in range(params['num_conv_layers']):
        classifier.add(Conv2D(params['num_filters'], (3, 3), padding='same',
                              kernel_regularizer=regularizers.l2(0.01)))
        classifier.add(BatchNormalization())
        classifier.add(ELU())
        classifier.add(Dropout(0.5))
        
    classifier.add(Flatten())
    
    for _ in range(params['num_dense_layers']):
        classifier.add(Dense(params['dense_layer_units'], activation='relu'))
        
    classifier.add(Dense(num_classes, activation='softmax'))
    
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return classifier
def genetic_algorithm(population_size, num_generations, mutation_rate, search_space, input_shape, num_classes, X_train, y_train, X_test, y_test):
    population = initialize_population(population_size, search_space)
    for generation in range(num_generations):
        print(f"Generation {generation + 1}/{num_generations}")
        fitness_scores = []
        for individual in population:
            fitness = evaluate_classifier(individual, input_shape, num_classes, X_train, y_train, X_test, y_test)
            fitness_scores.append((individual, fitness))
        fitness_scores.sort(key=lambda x: x[1], reverse=True)
        print(f"Best accuracy in generation {generation + 1}: {fitness_scores[0][1]}")
        selected_parents = [individual for individual, _ in fitness_scores[:2]]
        next_generation = selected_parents
        while len(next_generation) < population_size:
            parent1, parent2 = random.sample(selected_parents, 2)
            child = crossover(parent1, parent2)
            child = mutate(child, mutation_rate, search_space)
            next_generation.append(child)
        population = next_generation
    best_individual, best_fitness = max(fitness_scores, key=lambda x: x[1])
    return best_individual, best_fitness
def initialize_population(population_size, search_space):
    population = []
    for _ in range(population_size):
        individual = {param: random.choice(values) for param, values in search_space.items()}
        population.append(individual)
    return population

def crossover(parent1, parent2):
    child = {}
    for param in parent1.keys():
        child[param] = random.choice([parent1[param], parent2[param]])
    return child

def mutate(individual, mutation_rate, search_space):
    for param in individual.keys():
        if random.random() < mutation_rate:
            individual[param] = random.choice(search_space[param])
    return individual

def evaluate_classifier(params, input_shape, num_classes, X_train, y_train, X_test, y_test):
    classifier = build_classifier(params, input_shape, num_classes)
    classifier.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)
    _, accuracy = classifier.evaluate(X_test, y_test, verbose=0)
    return accuracy

input_shape = (22, 4, 1)
num_classes = 4
split_index = int(0.8 * X.shape[0])  
X_train, X_test = tf.split(X, [split_index, X.shape[0] - split_index])
y_train, y_test = tf.split(y, [split_index, y.shape[0] - split_index])

best_individual, best_fitness = genetic_algorithm(population_size, num_generations, mutation_rate, search_space, input_shape, num_classes, X_train, y_train, X_test, y_test)
print("Best Classifier Architecture found:")
print(best_individual)
print("Best Accuracy:", "%.2f" %best_fitness)
ga_accs["ELU"]=best_fitness


# MLP

In [ ]:
import random
import tensorflow as tf
from tensorflow.keras import layers, models

search_space = {
    'num_dense_layers': range(1, 4),
    'dense_layer_units': [32, 64, 128]
}

population_size = 10
num_generations = 5
mutation_rate = 0.1

def create_modified_model(params, input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=input_shape))
    for _ in range(params['num_dense_layers']):
        model.add(layers.Dense(params['dense_layer_units'], activation='relu'))
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

def genetic_algorithm(population_size, num_generations, mutation_rate, search_space, input_shape, num_classes, X_train, y_train, X_test, y_test):
    population = initialize_population(population_size, search_space)
    for generation in range(num_generations):
        print(f"Generation {generation + 1}/{num_generations}")
        fitness_scores = []
        for individual in population:
            fitness = evaluate_modified_model(individual, input_shape, num_classes, X_train, y_train, X_test, y_test)
            fitness_scores.append((individual, fitness))
        fitness_scores.sort(key=lambda x: x[1], reverse=True)
        print(f"Best accuracy in generation {generation + 1}: {fitness_scores[0][1]}")
        selected_parents = [individual for individual, _ in fitness_scores[:2]]
        next_generation = selected_parents
        while len(next_generation) < population_size:
            parent1, parent2 = random.sample(selected_parents, 2)
            child = crossover(parent1, parent2)
            child = mutate(child, mutation_rate, search_space)
            next_generation.append(child)
        population = next_generation
    best_individual, best_fitness = max(fitness_scores, key=lambda x: x[1])
    return best_individual, best_fitness
def initialize_population(population_size, search_space):
    population = []
    for _ in range(population_size):
        individual = {param: random.choice(values) for param, values in search_space.items()}
        population.append(individual)
    return population

def crossover(parent1, parent2):
    child = {}
    for param in parent1.keys():
        child[param] = random.choice([parent1[param], parent2[param]])
    return child

def mutate(individual, mutation_rate, search_space):
    for param in individual.keys():
        if random.random() < mutation_rate:
            individual[param] = random.choice(search_space[param])
    return individual

def evaluate_modified_model(params, input_shape, num_classes, X_train, y_train, X_test, y_test):
    modified_model = create_modified_model(params, input_shape, num_classes)
    modified_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    modified_model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)
    loss, accuracy = modified_model.evaluate(X_test, y_test, verbose=0)
    return accuracy

input_shape = (22, 4)  
num_classes = 4  
split_index = int(0.8 * X.shape[0]) 
X_train, X_test = tf.split(X, [split_index, X.shape[0] - split_index])
y_train, y_test = tf.split(y, [split_index, y.shape[0] - split_index])

best_individual, best_fitness = genetic_algorithm(population_size, num_generations, mutation_rate, search_space, input_shape, num_classes, X_train, y_train, X_test, y_test)
print("Best Modified Model Architecture found:")
print(best_individual)
print("Best Accuracy:", "%.2f" %best_fitness)
ga_accs["MLP"]=best_fitness


# RNN

In [ ]:
import random
import tensorflow as tf
from tensorflow.keras import layers, models


search_space = {
    'num_units': [32, 64, 128],
    'num_layers': range(1, 4)
}

population_size = 10
num_generations = 5
mutation_rate = 0.1

def create_rnn_model(params, input_shape, num_classes):
    model = models.Sequential()
    model.add(layers.SimpleRNN(params['num_units'], activation='relu', input_shape=input_shape))
    
    for _ in range(params['num_layers']):
        model.add(layers.Dense(params['num_units'], activation='relu'))
        
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model
def genetic_algorithm(population_size, num_generations, mutation_rate, search_space, input_shape, num_classes, X_train, y_train, X_test, y_test):
    population = initialize_population(population_size, search_space)
    for generation in range(num_generations):
        print(f"Generation {generation + 1}/{num_generations}")
        fitness_scores = []
        for individual in population:
            fitness = evaluate_rnn_model(individual, input_shape, num_classes, X_train, y_train, X_test, y_test)
            fitness_scores.append((individual, fitness))
        fitness_scores.sort(key=lambda x: x[1], reverse=True)
        print(f"Best accuracy in generation {generation + 1}: {fitness_scores[0][1]}")
        selected_parents = [individual for individual, _ in fitness_scores[:2]]
        next_generation = selected_parents
        while len(next_generation) < population_size:
            parent1, parent2 = random.sample(selected_parents, 2)
            child = crossover(parent1, parent2)
            child = mutate(child, mutation_rate, search_space)
            next_generation.append(child)
        population = next_generation
    best_individual, best_fitness = max(fitness_scores, key=lambda x: x[1])
    return best_individual, best_fitness
def initialize_population(population_size, search_space):
    population = []
    for _ in range(population_size):
        individual = {param: random.choice(values) for param, values in search_space.items()}
        population.append(individual)
    return population

def crossover(parent1, parent2):
    child = {}
    for param in parent1.keys():
        child[param] = random.choice([parent1[param], parent2[param]])
    return child

def mutate(individual, mutation_rate, search_space):
    for param in individual.keys():
        if random.random() < mutation_rate:
            individual[param] = random.choice(search_space[param])
    return individual

def evaluate_rnn_model(params, input_shape, num_classes, X_train, y_train, X_test, y_test):
    rnn_model = create_rnn_model(params, input_shape, num_classes)
    rnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    rnn_model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)
    loss, accuracy = rnn_model.evaluate(X_test, y_test, verbose=0)
    return accuracy

input_shape = (22, 4) 
num_classes = 4  
split_index = int(0.8 * X.shape[0]) 
X_train, X_test = tf.split(X, [split_index, X.shape[0] - split_index])
y_train, y_test = tf.split(y, [split_index, y.shape[0] - split_index])

best_individual, best_fitness = genetic_algorithm(population_size, num_generations, mutation_rate, search_space, input_shape, num_classes, X_train, y_train, X_test, y_test)
print("Best RNN Model Architecture found:")
print(best_individual)
print("Best Accuracy:", "%.2f" %best_fitness)
ga_accs["RNN"]=best_fitness


In [2]:
print(ga_accs)

{'CNN': 0.962, 'ELU': 0.955, 'MLP': 0.966, 'RNN': 0.958}
